# Error Analysis

Truncation & Discretization error

In [ ]:
import numpy as np
from scipy.stats import norm
import math
import matplotlib.pyplot as plt
import random 

European call option parameters

In [ ]:
S_0 = 100 # asset price at t=0 
sigma = 0.2 # volatility
K = 50 # strike price
r = 0.05 # risk-free interest rate
T = 1 # time to maturity in years

parameters = {'S_0': S_0, 
              'sigma': sigma,
              'K': K,
              'r': r,
              'T': T
              }

Black-Scholes Formula

In [ ]:
d_1 = (np.log(S_0/K)+(r+0.5*sigma**2)*T)/(sigma*T**0.5)
d_2 = (np.log(S_0/K)+(r-0.5*sigma**2)*T)/(sigma*T**0.5)

C_0 = S_0*norm.cdf(d_1)-math.exp(-r*T)*K*norm.cdf(d_2)
P_0 = math.exp(-r*T)*K*norm.cdf(-d_2)-S_0*norm.cdf(-d_1)

print(f'European Call (BS): {C_0}')
print(f'European Put (BS): {P_0}')

In [ ]:
def discretized_standard_MC(M, parameters, distribution, option_type):
    """
    Perform Monte Carlo estimation for option 
    pricing using a discretized approach.

    Parameters:
    M (int): Number of samples.
    parameters (dict): Dictionary containing option parameters.
    distribution (tuple): Tuple containing: probability distribution
    option_type (str): Type of the option, either 'call' or 'put'.

    Returns:
    tuple: A tuple containing the discounted expected payoff
        and the standard deviation of the payoff.
    """

    S_0 = parameters['S_0']
    r = parameters['r']
    sigma = parameters['sigma']
    T = parameters['T']
    K = parameters['K']

    x = distribution[0]
    y = distribution[1]

     # Generate M samples of the option payoff based on the option type
    if option_type == 'call':
        samples = [max(S_0*math.exp((r-sigma**2/2)*T+sigma*random.choices(x, y)[0])-K,0) for _ in range(M)]
    
    elif option_type == 'put':
        samples = [max(K-S_0*math.exp((r-sigma**2/2)*T+sigma*random.choices(x, y)[0]),0) for _ in range(M)]
    
    else:
        return -1

    # expected payoff
    mean = np.mean(samples)

    # standard deviation
    std = np.std(samples)

    # discounted payoff
    return math.exp(-r*T)*(mean), std 

In [ ]:
def standard_MC(M, parameters, option_type):
    """
    Perform standard Monte Carlo estimation for option 
    pricing

    Parameters:
    M (int): Number of samples.
    parameters (dict): Dictionary containing option parameters.
    option_type (str): Type of the option, either 'call' or 'put'.

    Returns:
    tuple: A tuple containing the discounted expected payoff
        and the standard deviation of the payoff.
    """

    S_0 = parameters['S_0']
    r = parameters['r']
    sigma = parameters['sigma']
    T = parameters['T']
    K = parameters['K']

    if option_type == 'call':
        samples = [max(S_0*math.exp((r-sigma**2/2)*T+sigma*np.random.normal(0, T**0.5))-K,0) for _ in range(M)]

    else:
        return -1

    # expected payoff
    mean = np.mean(samples)
    std = np.std(samples)

    # discounted payoff
    return math.exp(-r*T)*(mean), std 

Run experiment with different truncation and discretization

In [ ]:
option_type = 'call'
max_iterations = 1
max_samples = 22
max_discretization = 6
M_list = [2**(i+1) for i in range(max_samples)]
trunc_level = 3

for n in range(1, max_discretization+1):

    # create discrete probability distribution
    mean = 0 
    std = T**0.5
    x_discretized = np.linspace(-trunc_level*std, trunc_level*std, 2**n)
    y_discretized = [1/(std*(2*math.pi)**0.5)*math.exp(-1/2*((x-mean)/std)**2) for x in x_discretized]
    y_probability = [y/sum(y_discretized) for y in y_discretized]
    distribution = [x_discretized, y_probability]
    
    # save results
    results = []

    # repeat experiment
    for experiment in range(max_iterations):

        estimations = []
        standard_deviations = []
        errors = []
        
        # run with diffrent number of samples 
        for M in M_list:
            res = discretized_standard_MC(M, parameters, distribution, option_type)
            estimations.append(res[0])
            standard_deviations.append(res[1])
            errors.append(abs(res[0]-C_0)/C_0)
        results.append([estimations, standard_deviations, errors, M_list])

print(C_0)